# Import data

In [ ]:
import pandas as pd
file_path = "ESS11.dta"
df = pd.read_stata(file_path,convert_categoricals=False)

In [ ]:
df

In [ ]:
# drop y nan
df = df.dropna(subset=['loneliness']).reset_index(drop=True)
df

In [ ]:
df.columns.tolist()

In [ ]:

continuous_variables = [
    "SocTrust",
    "sclmeet",
    "stflife",
    "happy",
    "SRH",
    "InstitTrust",
    "rlgdgr",
    "religiousAttendance",
    "pray_r",
    "Nchildren",
    "education_con",
    "inprdsc",
    "sclact",
]

categorical_variables = [

    "hlthhmp",
    "depressed",
    "informalcare",


    "discrimination",
    "cnfpplh",

    "generation",
    "ethnicity",
    "gender",
    #"nobingnd",
    "age_groups",
    "domicil",
    "qualityHouse",
    #"region",
    "cntry_",
    #"country_group",
    "maritalstatus",
  #  "childrenbelow3_dummy", # Sept updated, delete this variables
    "childless",
    "partnerhh",
    "fnsdfml",
    "eiscedf",
    "eiscedm",
    "emprf14",
    "occf14b",
    "emprm14",
    "eisced",
    "employment",
    "EQHHincome_cat",
    "esec9",
    "subjhinc",
   "edu_mobility" # Sept updated, no need?
]

In [ ]:
selected_vars_old = continuous_variables + categorical_variables



In [ ]:
# 旧 -> 新 的映射关系
rename_dict = {
    "depressed": "Depressed",
    "partnerhh": "Partnerhh",
    "happy": "Happiness",
    "cntry_": "Country",
    "stflife": "LifeSatisfaction",
    "cnfpplh":"FamilyConflict",
    
    "maritalstatus": "MaritalStatus",
    "inprdsc": "SocInt",
    "subjhinc": "SubjHHincome",
    "InstitTrust":"InstitTrust",
    "sclmeet": "SocMeet",
    "Nchildren": "Nchildren",
    
    "fnsdfml": "CHFinancSit",

    "SocInt": "SocInt",

    "InstitTrust": "InstitTrust",
    "religiousAttendance":"ReligiousAtt",
    "generation": "MigBackground",
    "sclact": "SocAct",
    "gender": "Gender",
    "rlgdgr": "Religious",
    "age_groups": "AgeGroups",
    "pray_r": "Pray",
    "qualityHouse": "QualityHouse",
    
    "domicil": "Urbanity",
    "childless": "Childless",
    "SRH": "SRH",
    "SocTrust": "SocTrust",
    
    "occf14b": "CHFatherOcc",
 #   "occm14b": "CHMotherOcc",  
    "education_con": "YearsEducation",  
    
    "eiscedm": "EducationMother",
    "eiscedf": "EducationFather",
    
    "hlthhmp": "HealthHamp",
    
    "EQHHincome_cat": "Hhincome",
    
    "esec9":"OccStatus",
    
    "emprf14": "CHFatherEmploy",
    "emprm14": "CHMotherEmploy",
    
    "discrimination": "Discrimination",
    
    "ethnicity": "Ethnicity",
    "employment": "Employ",
    "informalcare": "InformalCare",
    "eisced": "EduDegree",
    
    "edu_mobility":"EduMobility"
      
}

df = df.rename(columns=rename_dict)

continuous_variables_new = [rename_dict.get(var, var) for var in continuous_variables]
categorical_variables_new = [rename_dict.get(var, var) for var in categorical_variables]

print(continuous_variables_new)
print(categorical_variables_new)


In [ ]:
selected_vars_new = continuous_variables_new + categorical_variables_new

In [ ]:
df[selected_vars_new].isnull().mean().sort_values(ascending=False)

In [ ]:
result = {var: df[var].nunique() for var in categorical_variables_new if var in df.columns}
filtered = {k: v for k, v in result.items() if v >= 6}

variables_with_many_classes = list(filtered.keys())
filtered


In [ ]:
variables_with_many_classes

In [ ]:
# 删除 country == 31 的行
df = df[df["Country"] != 31]

# 重置索引
df = df.reset_index(drop=True)

# 查看结果
print(df["Country"].unique())  # 检查是否已删除


In [ ]:
df

In [ ]:
categorical_variables_new = [var for var in categorical_variables_new if var not in variables_with_many_classes]

for var in variables_with_many_classes:
    if var not in continuous_variables_new:
        continuous_variables_new.append(var)


In [ ]:
selected_vars_new = continuous_variables_new + categorical_variables_new

df = df[selected_vars_new + ['loneliness'] + ['anweight']]

In [ ]:
df.shape

In [ ]:
columns_needed = ["loneliness", "Gender", "AgeGroups", "SubjHHincome", "CHFinancSit", "MigBackground", "Country",
                 "anweight"]

df_selected = df[columns_needed]

df_selected.to_stata("selected_data.dta", write_index=False)



# Models

In [ ]:
df

In [ ]:
import pandas as pd
import xgboost as xgb
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score, classification_report

target = "loneliness"   # 你的标签列名

X = df.drop(columns=[target, "anweight"])
y = df[target]

for col in categorical_variables_new:
    X[col] = X[col].astype("category")


In [ ]:
X.dtypes

In [ ]:
from sklearn.model_selection import train_test_split, GridSearchCV
from sklearn.metrics import accuracy_score
import xgboost as xgb

X_train, X_test, y_train, y_test = train_test_split(
    X, y, test_size=0.2, random_state=42, stratify=y
)

xgb_model = xgb.XGBClassifier(
    tree_method="hist",
    enable_categorical=True,
    random_state=42,
)

param_grid = {
    "n_estimators": [100, 200, 300],
    "learning_rate": [0.01, 0.05, 0.1],
    "max_depth": [4, 6, 8],
}

# 使用 GridSearchCV 搜索最佳超参数
grid_search = GridSearchCV(
    estimator=xgb_model,
    param_grid=param_grid,
    cv=5,
    scoring='roc_auc',  # 用 AUC 评估
    n_jobs=-1,
    verbose=2  #
)

grid_search.fit(X_train, y_train)

print("Best parameters:", grid_search.best_params_)

# 用最佳模型进行预测
best_model = grid_search.best_estimator_


In [ ]:
# 预测类别
from sklearn.metrics import accuracy_score, roc_auc_score, precision_score, recall_score, f1_score

y_pred = best_model.predict(X_test)

y_pred_proba = best_model.predict_proba(X_test)[:, 1]

accuracy = accuracy_score(y_test, y_pred)
auc = roc_auc_score(y_test, y_pred_proba)
f1 = f1_score(y_test, y_pred)

print(f"Accuracy: {accuracy:.4f}")
print(f"AUC: {auc:.4f}")

print(f"F1-score: {f1:.4f}")


# SHAP


In [ ]:
import pandas as pd
import numpy as np
import shap

# 计算 SHAP 值
explainer = shap.TreeExplainer(best_model)
shap_values = explainer.shap_values(X_test)

# 计算每个特征的平均绝对 SHAP 值
shap_importance = pd.DataFrame({
    'feature': X_test.columns,
    'importance': np.abs(shap_values).mean(axis=0)
})

# 按重要性排序
shap_importance = shap_importance.sort_values(by='importance', ascending=False)
print(shap_importance)


In [ ]:
import matplotlib.pyplot as plt

# 选择前N个最重要特征
top_n = 40
top_features = shap_importance.head(top_n)

# 绘制条形图
plt.figure(figsize=(8,13))
plt.barh(top_features['feature'][::-1], top_features['importance'][::-1], color='skyblue')
plt.xlabel('Mean |SHAP value|')
plt.title(f'Top {top_n} Feature Importance (SHAP)')
plt.show()


## Plot age country

In [ ]:
file_path = "ESS11.dta"
get_country_df = pd.read_stata(file_path,convert_categoricals=False)
get_country_df = get_country_df.dropna(subset=['loneliness']).reset_index(drop=True)

In [ ]:
age_group_name=['15-29','30-49','50-64','65-80', '81+']


In [ ]:
# plot age 


import pandas as pd
import numpy as np
import shap
import matplotlib.pyplot as plt
import os

top_n = 40

save_dir = './'
os.makedirs(save_dir, exist_ok=True)  # 如果目录不存在就创建


for i in range(5):  # 跳过 NaN

    subset = X_test[X_test['AgeGroups'] == (i+1)]

    # 计算 SHAP 值
    shap_values_subset = explainer.shap_values(subset)

    # 计算平均绝对 SHAP
    shap_importance_subset = pd.DataFrame({
        'feature': subset.columns,
        'importance': np.abs(shap_values_subset).mean(axis=0)
    })

    # 排除 loop_variables

    # 取前 top_n 特征
    top_features = shap_importance_subset.sort_values(by='importance', ascending=False).head(top_n).copy()


    # 保存到总表

    plt.figure(figsize=(8,10))
    plt.barh(top_features['feature'][::-1], top_features['importance'][::-1], color='skyblue')
    plt.xlabel('Mean |SHAP value|')
    plt.title(f"Top {top_n} Feature Importance\n AgeGroups = {age_group_name[i]}")
    plt.tight_layout()

    # 保存图像
    filename = f"AgeGroups_{i+1}.png"
    filepath = os.path.join(save_dir, filename)
    plt.savefig(filepath, dpi=300)  # 保存高分辨率图片
    print('save a figure')
    plt.close()  # 关闭当前 figure，防止内存占用


In [ ]:
## Plot country

In [ ]:
group_country_dict = (
    get_country_df
    .groupby('country_group')['cntry_']
    .unique()
    .apply(list)
    .to_dict()
)

print(group_country_dict)


In [ ]:
group_country_dict[1]

In [ ]:
country_group_name = ['Eastern European','Southern European','Nordic', 'Western European']

In [ ]:
for i in range(4):
    
    
    subset = X_test[X_test['Country'].isin(group_country_dict[i+1])]
    
   # 计算 SHAP 值
    shap_values_subset = explainer.shap_values(subset)

    # 计算平均绝对 SHAP
    shap_importance_subset = pd.DataFrame({
        'feature': subset.columns,
        'importance': np.abs(shap_values_subset).mean(axis=0)
    })


    top_features = shap_importance_subset.sort_values(by='importance', ascending=False).head(top_n).copy()


    plt.figure(figsize=(8,10))
    plt.barh(top_features['feature'][::-1], top_features['importance'][::-1], color='skyblue')
    plt.xlabel('Mean |SHAP value|')
    plt.title(f"Top {top_n} Feature Importance\n Country = {country_group_name[i]}")
    plt.tight_layout()

    # 保存图像
    filename = f"Country_{i+1}.png"
    filepath = os.path.join(save_dir, filename)
    plt.savefig(filepath, dpi=300)  # 保存高分辨率图片
    print('save a figure')
    plt.close()  # 关闭当前 figure，防止内存占用

In [ ]:
# For Gender 

In [ ]:
gender_name = ['Men', 'Women']
gender_values = [0.0, 1.0]  # 对应 Gender 列的实际值

for i, gender_val in enumerate(gender_values):
    subset = X_test[X_test['Gender'] == gender_val]
    if subset.empty:
        continue

    # 对二分类模型取类别 1 的 SHAP 值
    shap_values_subset = explainer.shap_values(subset)
    if isinstance(shap_values_subset, list):
        shap_values_subset = shap_values_subset[1]

    shap_importance_subset = pd.DataFrame({
        'feature': subset.columns,
        'importance': np.abs(shap_values_subset).mean(axis=0)
    })

    top_features = shap_importance_subset.sort_values(by='importance', ascending=False).head(top_n)

    plt.figure(figsize=(8,10))
    plt.barh(top_features['feature'][::-1], top_features['importance'][::-1], color='skyblue')
    plt.xlabel('Mean |SHAP value|')
    plt.title(f"Top {top_n} Feature Importance\n Gender = {gender_name[i]}")
    plt.tight_layout()

    filepath = os.path.join(save_dir, f"Gender_{int(gender_val)}.png")
    plt.savefig(filepath, dpi=300)
    print(f'Saved figure: {filepath}')
    plt.close()
